In [96]:
import requests
import json

In [120]:
class Track:

    def __init__(self, id, name, artist):
        self.id = id
        self.name = name
        self.artist = artist
    
    def create_spotify_uri(self):
        return f"spotify:track:{self.id}"
    
    def __str__(self):
        return self.name + " by " + self.artist
class Playlist:
    # "Spotify playlist class"


    # ID: playlist ID | Name: playlist Name 
    def __init__(self, id, name):
        self.id = id
        self.name = name
    
    def __str__(self):
        return f"Playlist: {self.name}"

# Where user interacts with the spotify client to obtain user_id and authorization token 
import requests
import json
from track import Track

class SpotifyClient:

    def __init__(self, oauth_token, user_id):
        self.__oauth_token = oauth_token
        self.__user_id = user_id
    
    def get_current_user_recent_tracks(self, limit = 10):
        url = f"https://api.spotify.com/v1/me/player/recently-played?limit={limit}"
        response = self.get_api_requests(url)
        response_json = response.json()
        tracks = [Track(track['track']['id'], track['track']['name'], track['track']['artists'][0]['name']) for track in response_json['items']]
        return tracks
    
    def get_recommended_tracks(access_token, tracks, limit = 20):
        seed_tracks_url = ""
        for seedTrack in tracks:
            seed_tracks_url += seedTrack.id + ","
        seed_tracks_url = seed_tracks_url[:-1]# gets rid of the ending comma
        url = f"https://api.spotify.com/v1/recommendations?seed_tracks={seed_tracks_url}&limit={limit}"
        response = requests.get(
            url,
            headers = {
                "Content-type": "application/json",
                "Authorization": f"Bearer {access_token}"
            }
        )
        response_json = response.json()
        """
        FOR SOME REASON BREAKING. FIX IT
        """
        # recc_tracks = [Track(track['id'], track['name'], track['artists'][0]['name']) for track in response_json['tracks']]
        return recc_tracks

    def create_playlist(self, access_token, playlistName):
        data = json.dumps({
                "name": playlistName,
                "description": "Recommended songs",
                "public": True
        })
        url = f"https://api.spotify.com/v1/users/{self._user_id}/playlists"
        response = requests.post(
            url, 
            data,
            headers = {
                    "Content-Type": "application/json",
                    "Authorization": f"Bearer {access_token}"
                }
        )
        response_json = response.json()
        playlistID = response_json['id']
        playlist = Playlist(playlistID, playlistName)
        return playlist
        
    def add_items_to_playlist(access_token, playlist, tracks): 
        track_url_list = [track.create_spotify_uri() for track in tracks]
        data = json.dumps(track_url_list)
        url = f"https://api.spotify.com/v1/playlists/{playlist.id}/tracks"
        response = requests.post(
            url, 
            data,
            headers = {
                    "Content-Type": "application/json",
                    "Authorization": f"Bearer {access_token}"
                }
        )
        response_json = response.json()
        return response_json
    def get_api_request(self, url):
        response = requests.get(
            url, 
            headers = {
                "Content-Type": "application/json",
                "Authorization": f"Bearer {self.__oauth_token}"
            }
        )
        return response

    def post_api_request(self, url, data):
        response = requests.post(
            url,
            data = data,
            headers = {
                "Content-Type": "application/json",
                "Authorization": f"Bearer {self.__oauth_token}"
            }
        )
        return response

authorizationToken = input("Please enter your authorization token: ")
userID = input("Please enter in your userID: ")
spotify = SpotifyClient(authorizationToken, userID)
print(spotify)
